In [1]:
%%spark
// Fabric config
var WorkspaceId = "<workspace_id>"
var LakehouseId = "<lakehouse_id>"
var IntermediateFolderPath = f"abfss://${WorkspaceId}@onelake.dfs.fabric.microsoft.com/${LakehouseId}/Files/hms_output/syn/"

var ContainerName = "<container_name>"
var StorageName = "<storage_name>"
var SynapseWorkspaceName = <synapse_workspace_name>
var WarehouseMappings:Map[String, String] = Map(
    f"abfss://${ContainerName}@${StorageName}.dfs.core.windows.net/synapse/workspaces/${SynapseWorkspaceName}/warehouse"-> f"abfss://${WorkspaceId}@onelake.dfs.fabric.microsoft.com/${LakehouseId}/Files/warehouse_dir_syn",
    f"dbfs:/mnt/${StorageName}/databricks/warehouse"->f"abfss://${WorkspaceId}@onelake.dfs.fabric.microsoft.com/${LakehouseId}/Files/warehouse_dir_dbx",
    f"abfss://${ContainerName}@${StorageName}.dfs.core.windows.net/apps/spark/warehouse"->f"abfss://${WorkspaceId}@onelake.dfs.fabric.microsoft.com/${LakehouseId}/Files/warehouse_dir_hdi"
)

// Metastore config
var DatabasePrefix = ""
var TablePrefix = ""
var IgnoreIfExists = true

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 3, Finished, Available)

WorkspaceId: String = d3de3f63-266f-454c-87e1-5c7c598a41dd
LakehouseId: String = 255fdd58-6a1d-40b6-86f7-76102aab0fbd
IntermediateFolderPath: String = abfss://d3de3f63-266f-454c-87e1-5c7c598a41dd@onelake.dfs.fabric.microsoft.com/255fdd58-6a1d-40b6-86f7-76102aab0fbd/Files/hms_output/syn/
ContainerName: String = default
StorageName: String = stmurggudata
WarehouseMappings: Map[String,String] = Map(abfss://default@stmurggudata.dfs.core.windows.net/synapse/workspaces/syn-murggu-hms/warehouse -> abfss://d3de3f63-266f-454c-87e1-5c7c598a41dd@onelake.dfs.fabric.microsoft.com/255fdd58-6a1d-40b6-86f7-76102aab0fbd/Files/warehouse_dir_syn, dbfs:/mnt/stmurggudata/databricks/warehouse -> abfss://d3de3f63-266f-454c-87e1-5c7c598a41dd@onelake.dfs.fabric.microsoft.com/255fdd58-6a1d-40b6-86f7-76102aab0fbd/Files/warehouse_dir_dbx, abfss://default@stmurggudata.dfs.core.windows.net/apps/spark/warehouse -> abfss://d3de3f63-266f-454c-87e1-5c7c598a41dd@onelake.dfs.fabric.microsoft.com/255fdd58-6a1d-40b6-86f7-7

In [4]:
%%spark
import java.net.URI
import java.util.Calendar

import scala.collection.mutable.{ListBuffer, Map}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{ObjectType, _}
import org.apache.spark.sql.catalyst._
import org.apache.spark.sql.catalyst.catalog._
import org.json4s._
import org.json4s.JsonAST.JString
import org.json4s.jackson.Serialization
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.http.client.methods.{CloseableHttpResponse, HttpPost}
import org.apache.http.entity.StringEntity
import org.apache.http.impl.client.{CloseableHttpClient, HttpClients}
import scala.io.Source


var locationPrefixMappingList = WarehouseMappings.toList.sortBy(pair => pair._1).reverse

val DatabaseType = "database"
val TableType = "table"
val PartitionType = "partition"

object ImportMetadata {

  val spark = SparkSession.builder().getOrCreate()

  case object URISerializer extends CustomSerializer[URI](format => ( {
    case JString(uri) => new URI(uri)
  }, {
    case uri: URI => JString(uri.toString())
  }))

  case object SturctTypeSerializer extends CustomSerializer[StructType](format => ( {
    case JString(structType)  => DataType.fromJson(structType).asInstanceOf[StructType]
  }, {
    case structType: StructType => JString(structType.json)
  }))


  implicit val formats = DefaultFormats + URISerializer + SturctTypeSerializer

  case class CatalogPartitions(database: String, table: String, tablePartitons: Seq[CatalogTablePartition])

  case class CatalogTables(database: String, tables: Seq[CatalogTable])

  case class CatalogStat(entityType: String, count: Int, database: Option[String], table: Option[String])

  def ConvertLocation(location: String) : String = {
    var locationMapping = locationPrefixMappingList.find(mapping => {location.startsWith(mapping._1)})

    if (locationMapping != None) {
      return location.replaceFirst(locationMapping.get._1, locationMapping.get._2)
    }

    return location;
  }

  def ConvertCatalogDatabase(database: CatalogDatabase) : CatalogDatabase = {
    var convertedDatabase  = new CatalogDatabase(
      DatabasePrefix + database.name,
      database.description,
      new URI(ConvertLocation(database.locationUri.toString())),
      database.properties)

    return convertedDatabase;
  }

  def ConvertCatalogStorageFormat(format : CatalogStorageFormat) : CatalogStorageFormat = {

    var formatlocation: Option[URI] = None
    if (format.locationUri != None) {
      formatlocation = Some(new URI(ConvertLocation(format.locationUri.get.toString())))
    }

    var convertedStorageFormat = new CatalogStorageFormat(
      formatlocation,
      format.inputFormat,
      format.outputFormat,
      format.serde,
      format.compressed,
      format.properties
    )

    return  convertedStorageFormat;
  }

  def ConvertCatalogTable(table: CatalogTable) : CatalogTable = {

    var dbName = Some(DatabasePrefix + table.identifier.database.get);
    var tblName = TablePrefix + table.identifier.table;

    var convertedTable = new CatalogTable(
      new TableIdentifier(tblName, dbName),
      org.apache.spark.sql.catalyst.catalog.CatalogTableType("EXTERNAL"),
      ConvertCatalogStorageFormat(table.storage),
      table.schema,
      table.provider,
      table.partitionColumnNames,
      table.bucketSpec,
      table.owner,
      table.createTime,
      table.lastAccessTime,
      table.createVersion,
      table.properties,
      table.stats,
      table.viewText,
      table.comment,
      table.unsupportedFeatures,
      table.tracksPartitionsInCatalog,
      table.schemaPreservesCase,
      table.ignoredProperties)

    return convertedTable;
  }

  def ConvertCatalogTablePartition(partition : CatalogTablePartition) : CatalogTablePartition = {
    var convertedPartition = new CatalogTablePartition(
      partition.spec,
      ConvertCatalogStorageFormat(partition.storage),
      partition.parameters,
      partition.createTime,
      partition.lastAccessTime,
      partition.stats
    );

    return convertedPartition;
  }

  val MaxRetryCount = 3;

  def RetriableFunc(func: () => Unit, retryCount: Int = 0): Unit = {
    try {
      func()
    } catch {
      case e:Exception => {
        if (retryCount < MaxRetryCount){
          RetriableFunc(func, retryCount + 1)
        } else {
          throw e
        }
      }
    }
  }

  def RetriableQueryFunc(func: () => Object, retryCount: Int = 0): Object = {
    try {
      func()
    } catch {
      case e:Exception => {
        if (retryCount < MaxRetryCount){
          RetriableQueryFunc(func, retryCount + 1)
        } else {
          throw e
        }
      }
    }
  }

// Create DBs

  def CreateDatabases(dataPath: String): Int = {

    println("Start to create databases " + Calendar.getInstance().getTime())

    val ds = spark.read.format("text").load(dataPath)

    var createdCount = 0;
    var existsDbs = spark.sharedState.externalCatalog.listDatabases()
    var data = ds.collect()
    var total = data.size

    data.foreach(row => {
      var jsonString = row.getString(0)
      var newDb = ConvertCatalogDatabase(Serialization.read[CatalogDatabase](jsonString))

      var exists = existsDbs.contains(newDb.name)
      if (exists && !IgnoreIfExists) {

        println(createdCount + "/" + total + " databases created. " + Calendar.getInstance().getTime())
        println("Database " + newDb.name + " already exists")

        throw new Exception("Database " + newDb.name + " already exists")
      } else if (!exists) {
        CreateDatabase(newDb.name)
      }

      createdCount+=1;

      if (createdCount%100 == 0) {
        println(createdCount + "/" + total + " databases created" + Calendar.getInstance().getTime())
      }
    });

    println("Databases Created completed. Total " + createdCount + " database created. " + Calendar.getInstance().getTime())
    return createdCount
  }

  def CreateDatabase(dbName: String) = {
    mssparkutils.lakehouse.create(dbName, "imported db", WorkspaceId)
  }

  // Create Tables

  def CreateTables(dataPath: String): Int = {
    println("Start to create tables " + Calendar.getInstance().getTime())

    val ds = spark.read.format("text").load(dataPath);

    var createdCount = 0;
    ds.collect().foreach(row => {
      var jsonString = row.getString(0)
      var tables = Serialization.read[CatalogTables](jsonString);

      var existsTables = spark.sharedState.externalCatalog.listTables(DatabasePrefix + tables.database)
      var perTables = tables.tables.toParArray

      perTables.foreach(table => {
        var newTable = ConvertCatalogTable(table)
        var exists = existsTables.contains(newTable.identifier.table)
        if (exists && !IgnoreIfExists) {

          println(createdCount + " tables created. " + Calendar.getInstance().getTime())
          println("Table " + newTable.identifier.database + "." + newTable.identifier.table + " already exists")

          throw new Exception("Table " + newTable.identifier.database + "." + newTable.identifier.table + " already exists")
        } else if (!exists) {
          CreateTable(newTable)
        }

        createdCount+=1;
      })

      println(createdCount + " tables created" + Calendar.getInstance().getTime())
    })

    println("Tables Created completed. Total " + createdCount + " table created. " + Calendar.getInstance().getTime())
    return createdCount
  }

  def CreateTable(table:CatalogTable) = {
    RetriableFunc(() => {
      spark.sharedState.externalCatalog.createTable(table, IgnoreIfExists)
    })
  }

  def ValidateTablePath(dataPath: String) = {
    println("Start to validate table path " + Calendar.getInstance().getTime())

    val ds = spark.read.format("text").load(dataPath)
    var hadoopConf = spark.sparkContext.hadoopConfiguration

    ds.collect().foreach(row => {
      var jsonString = row.getString(0)
      var tables = Serialization.read[CatalogTables](jsonString);

      tables.tables.toParArray.foreach(table => {
        var newTable = ConvertCatalogTable(table)
        try{
          var p = new Path(newTable.location);
          var fs = p.getFileSystem(hadoopConf);
        } catch {
          case e:Exception => {
            throw new Exception("Validate table path failed. Table: " + newTable.identifier.database.getOrElse() + "." + newTable.identifier.table + ", Location: " +  newTable.location + " , exception: " + e)
          }
        }
      })
    })

    println("Validate table path completed")
  }

  // Create Partitions

  def CreatePartitions(dataPath: String): Int = {
    println("Start to create partitions " + Calendar.getInstance().getTime())

    val ds = spark.read.format("text").load(dataPath);

    var createdCount = 0;
    ds.collect().foreach(row => {
      var jsonString = row.getString(0)
      var parts = Serialization.read[CatalogPartitions](jsonString);

      var catalogTablePartitions = new ListBuffer[CatalogTablePartition]()
      parts.tablePartitons.foreach( part => {
        catalogTablePartitions += ConvertCatalogTablePartition(part)
      })

      RetriableFunc(() => {
        spark.sharedState.externalCatalog.createPartitions(DatabasePrefix + parts.database, TablePrefix + parts.table, catalogTablePartitions, IgnoreIfExists)
      })

      createdCount+=catalogTablePartitions.size;
      println(createdCount +  " partitions created" + Calendar.getInstance().getTime())
    });

    println("Partition Created completed. Total " + createdCount + " partition created. " + Calendar.getInstance().getTime())
    return createdCount
  }

  def LoadStats(dataPath: String): List[CatalogStat] = {
    println("Start to load stats " + Calendar.getInstance().getTime())

    val ds = spark.read.format("text").load(dataPath);

    var statBuffer = new ListBuffer[CatalogStat];
    ds.collect().foreach(row => {
      var jsonString = row.getString(0)
      statBuffer.append(Serialization.read[CatalogStat](jsonString))
    })

    return statBuffer.toList
  }

  def ValidateImportResult(entityType: String, createdCount: Int, stats: List[CatalogStat]):Boolean = {
    var mappingStat = stats.find(stat => stat.entityType == entityType && stat.database == None && stat.table == None);
    if (mappingStat == None) {
      println("Validated failed. Failed to get orignal " + entityType + " count")
      return false
    }

    if (mappingStat.get.count != createdCount) {
      println("Validated failed. Catalog object count missmatch. Expected " + entityType + " count is " + mappingStat.get.count + ", but created " + entityType + " count is " + createdCount);
      return false;
    }

    println("Validated passed. Catalog objects are created as expected. " + createdCount + " " + entityType + " are created." )
    return true
  }

  def ImportCatalogObjectsFromFile(inputPath: String) = {

    val dbsPath = inputPath + "databases";
    val tablesPath = inputPath + "tables";
    val partPath = inputPath + "partitions";

    CreateDatabases(dbsPath)
    CreateTables(tablesPath)
    CreatePartitions(partPath)
  }
}

var stats = ImportMetadata.LoadStats(IntermediateFolderPath + "/catalogObjectStats")

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 6, Finished, Available)

Start to load stats Tue Sep 26 11:02:53 UTC 2023
import java.net.URI
import java.util.Calendar
import scala.collection.mutable.{ListBuffer, Map}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{ObjectType, _}
import org.apache.spark.sql.catalyst._
import org.apache.spark.sql.catalyst.catalog._
import org.json4s._
import org.json4s.JsonAST.JString
import org.json4s.jackson.Serialization
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.http.client.methods.{CloseableHttpResponse, HttpPost}
import org.apache.http.entity.StringEntity
import org.apache.http.impl.client.{CloseableHttpClient, HttpClients}
import scala.io.Source
locationPrefixMappingList: List[(String, String)] = List((dbfs:/mnt/stmurggudata/databricks/warehouse,abfss://d3de3f63-266f-454c-87e1-5c7c598a41dd@onelake.dfs.fabric.microsoft.com/255fdd58-6a1d-40b6-86f7-76102aab0fbd/Files/warehouse_dir_dbx), (abfss://default@stmurggudata.dfs.core.windows.net/sy

In [5]:
%%spark
// Validate table path
ImportMetadata.ValidateTablePath(IntermediateFolderPath + "/tables")

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 7, Finished, Available)

Start to validate table path Tue Sep 26 11:03:07 UTC 2023
Validate table path completed


In [6]:
// Import Databases
var createdDb = ImportMetadata.CreateDatabases(IntermediateFolderPath + "/databases")
ImportMetadata.ValidateImportResult(DatabaseType, createdDb, stats)

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 8, Finished, Available)

Start to create databases Tue Sep 26 11:03:16 UTC 2023
Databases Created completed. Total 1 database created. Tue Sep 26 11:03:32 UTC 2023
Validated passed. Catalog objects are created as expected. 1 database are created.
createdDb: Int = 1
res13: Boolean = true


In [7]:
%%spark
// Validate Lakehouse (database) creation
spark.sharedState.externalCatalog.listDatabases()

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 9, Finished, Available)

res15: Seq[String] = Buffer(synapse, syndb1)


In [8]:
%%spark
// Import Tables
var createdTbl = ImportMetadata.CreateTables(IntermediateFolderPath + "/tables")
ImportMetadata.ValidateImportResult(TableType, createdTbl, stats)

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 10, Finished, Available)

Start to create tables Tue Sep 26 11:03:43 UTC 2023
16 tables createdTue Sep 26 11:04:02 UTC 2023
Tables Created completed. Total 16 table created. Tue Sep 26 11:04:02 UTC 2023
Validated passed. Catalog objects are created as expected. 16 table are created.
createdTbl: Int = 16
res17: Boolean = true


In [9]:
%%spark
// Import Partitions
var createdPart = ImportMetadata.CreatePartitions(IntermediateFolderPath + "/partitions")
ImportMetadata.ValidateImportResult(PartitionType, createdPart, stats)

StatementMeta(, 0c0ab054-b220-4fc9-9a59-833c571f737c, 11, Finished, Available)

Start to create partitions Tue Sep 26 11:04:06 UTC 2023
2 partitions createdTue Sep 26 11:04:09 UTC 2023
4 partitions createdTue Sep 26 11:04:10 UTC 2023
6 partitions createdTue Sep 26 11:04:10 UTC 2023
8 partitions createdTue Sep 26 11:04:11 UTC 2023
10 partitions createdTue Sep 26 11:04:12 UTC 2023
12 partitions createdTue Sep 26 11:04:12 UTC 2023
Partition Created completed. Total 12 partition created. Tue Sep 26 11:04:12 UTC 2023
Validated passed. Catalog objects are created as expected. 12 partition are created.
createdPart: Int = 12
res19: Boolean = true


In [9]:
%%sql
DESCRIBE EXTENDED syndb1.t_manag_part_delta

StatementMeta(, 10d88e4e-d3dd-415c-93ed-45b9586d38ca, 11, Finished, Available)

<Spark SQL result set with 22 rows and 3 fields>

In [12]:
%%pyspark
spark.catalog.listTables()

StatementMeta(, 10d88e4e-d3dd-415c-93ed-45b9586d38ca, 15, Finished, Available)

[Table(name='students_3', database='synapse', description=None, tableType='MANAGED', isTemporary=False)]